<a href="https://colab.research.google.com/github/navithaabhinaya/Daa1/blob/main/Glucoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from shutil import copy

# Define the path to your dataset directory
dataset_dir = '/content/gdrive/MyDrive/Gdataset/Glaucoma dataset.zip (Unzipped Files)/preprocessed_images'

# Define the path for the training and validation directories
train_dir = '/content/gdrive/MyDrive/trainG'
val_dir = '/content/gdrive/MyDrive/ValidG'

# Define the ratio for the validation set (e.g., 20%)
validation_split = 0.2

# Create the training and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# List all the image files in your dataset directory
image_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]

# Split the dataset into training and validation sets
train_files, val_files = train_test_split(image_files, test_size=validation_split, random_state=42)

# Copy the training set images to the training directory
for filename in train_files:
    src = os.path.join(dataset_dir, filename)
    dst = os.path.join(train_dir, filename)
    copy(src, dst)

# Copy the validation set images to the validation directory
for filename in val_files:
    src = os.path.join(dataset_dir, filename)
    dst = os.path.join(val_dir, filename)
    copy(src, dst)

print(f"Split dataset into {len(train_files)} training images and {len(val_files)} validation images.")


Split dataset into 862 training images and 216 validation images.


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [5]:
train_data_dir = '/content/gdrive/MyDrive/trainG'
val_data_dir = '/content/gdrive/MyDrive/ValidG'

In [8]:
num_classes = 2 # 'Glaucoma ' and 'non Glaucoma '
img_width, img_height = 224, 224 # MobileNetV2 input size
batch_size = 32
epochs = 30

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
rescale=1.0 / 255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='binary' # 'binary' for binary classification
)
validation_generator = val_datagen.flow_from_directory(
val_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='binary'
)

In [ ]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# Freeze the pre-trained layers
for layer in base_model.layers:
layer.trainable = False
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(
train_generator,
steps_per_epoch=train_generator.samples // batch_size,
epochs=epochs,
validation_data=validation_generator,
validation_steps=validation_generator.samples // batch_size
)

In [ ]:
 #Save the trained model
model.save('Glaucoma_detection_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('poacher_detection_model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
def preprocess_image(image_path):
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0 # Rescale to [0, 1]
return img

In [ ]:
new_image_paths = [
'/content/drive/MyDrive/val/val1/img231.jpg',
'/content/drive/MyDrive/val/val1/img228.jpg',
'/content/drive/MyDrive/val/val1/i2.jpg',
'/content/drive/MyDrive/val/val1/img208.jpg',
'/content/drive/MyDrive/val/val1/img202.jpg',
'/content/drive/MyDrive/train/train1/056.jpg'
]

In [ ]:
if prediction > 0.5:
  result = 'Glaucoma Detected'
  subject = 'Glaucoma Detected'

else:
  result = 'No Glaucoma Detected'
print(f"Image: {image_path} - Prediction: {result}")
